In [2]:
import os
import tempfile
from urllib.parse import urlparse

import matplotlib.pyplot as plt
# import planetary_computer
# import pystac

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 12)

import torch
from torch.utils.data import DataLoader

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [1]:
import fiona

In [3]:
from torchgeo.datasets import RasterDataset, stack_samples, unbind_samples

ImportError: dlopen(/opt/anaconda3/envs/py10/lib/python3.10/site-packages/rasterio/_base.cpython-310-darwin.so, 0x0002): Symbol not found: _ZSTD_compressBound
  Referenced from: <C8441800-341E-3CB0-9D0F-53907A12FCD4> /opt/anaconda3/envs/py10/lib/libgdal.30.dylib
  Expected in:     <8F9D674B-86E4-344E-B1CF-865220ACDD68> /opt/anaconda3/envs/py10/lib/libblosc.1.21.5.dylib

In [3]:
from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler

AttributeError: partially initialized module 'fiona' has no attribute '_loading' (most likely due to a circular import)